# Solutions to rule 1010 using gaussian elimination

In [1]:
import torch
import networkx as nx
import numpy as np
from tqdm import tqdm

In [2]:
def gauss_torch(M):

    m,n = M.shape

    i=0
    j=0

    while i < m and j < n:
        # find value and index of largest element in remainder of column j
        indexes=M[i:,j].nonzero()
        if indexes.numel()>0:
            k = indexes[0][0]+i
        else:
            k=i 
        # swap rows
        temp = torch.clone(M[k])
        M[k] = M[i]
        M[i] = temp
        aijn = M[i, j:]
        col = torch.clone(M[:, j]) #make a copy otherwise M will be directly affected
        col[i] = 0 #avoid changing pivot row with itself
        flip = torch.outer(col, aijn)
        M[:, j:] = M[:, j:] ^ flip
        i += 1
        j +=1

    return M

In [3]:
Ns=[80000]
rule=['1','0','1','0']
d=len(rule)-1
num_exp=10
found_for_N=np.zeros(len(Ns))
for idx, N in enumerate(Ns):
    found=0
    for exp in tqdm(range(num_exp)):
        # create adjacency matrix M
        G=nx.random_regular_graph(n=N, d=d)
        M=np.ones((N,N+1), dtype=bool)
        M[:,:N]=nx.adjacency_matrix(G).toarray()
        M[:,:N]+=np.eye(N, dtype=bool)
        M_torch=torch.tensor(M, device='cuda')
        sol=gauss_torch(M_torch)[:,N].cpu().numpy()
        sol=np.append(sol, False)
        num_errors=np.sum(np.sum(M*sol, axis=1)%2!=1)
        if num_errors==0:
            found+=1
    print('N = ', N,' num. sat. : ', found)
    found_for_N[idx]=found

100%|██████████| 10/10 [8:59:02<00:00, 3234.25s/it] 

N =  80000  num. sat. :  10
